In [12]:
import os
import numpy as np
import cv2
import torch
import h5py
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN,HDBSCAN,OPTICS
from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
from deeplsd.geometry.viz_2d import plot_images, plot_lines

# Load an image from the Hypersim dataset
image_path = '../assets/data/ai_001_001/images/scene_cam_00_final_preview/frame.0001.color.jpg'
img = cv2.imread(image_path)[:, :, ::-1]
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Load depth and normal maps
depth_path = '../assets/data/ai_001_001/images/scene_cam_00_geometry_hdf5/frame.0001.depth_meters.hdf5'
normal_path = '../assets/data/ai_001_001/images/scene_cam_00_geometry_hdf5/frame.0001.normal_world.hdf5'
position_path = '../assets/data/ai_001_001/images/scene_cam_00_geometry_hdf5/frame.0001.position.hdf5'

with h5py.File(depth_path, 'r') as f:
    depth_map = np.array(f['dataset'])
with h5py.File(normal_path, 'r') as f:
    normal_map = np.array(f['dataset'])
with h5py.File(position_path, "r") as f:
    world_coordinates = np.array(f["dataset"])

depth_map = depth_map.astype(np.float32)
normal_map = normal_map.astype(np.float32)
world_coordinates = world_coordinates.astype(np.float32)

# Initialize DeepLSD model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = {'detect_lines': True, 'line_detection_params': {'merge': False, 'filtering': True, 'grad_thresh': 3}}
ckpt = torch.load('../weights/deeplsd_md.tar', map_location='cpu', weights_only=False)
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()

# Detect lines
inputs = {'image': torch.tensor(gray_img, dtype=torch.float32, device=device)[None, None] / 255.}
with torch.no_grad():
    out = net(inputs)
    pred_lines = out['lines'][0]


In [15]:

import numpy as np

def compute_plane(points, normals):
    """
    Compute a artficial plane equation ax + by + cz + d = 0 
    for a set of 3D points and their corresponding normal vectors.

    Parameters:
    
    points (np.ndarray): Nx3 array of 3D points along a line.
    normals (np.ndarray): Nx3 array of normal vectors at each point.

        Returns:
        
    np.ndarray: Plane equation coefficients [a, b, c, d]."""
    if len(points) < 3:
        return None  # Not enough points to define a plane

    # Compute the average normal vector
    avg_normal = np.mean(normals, axis=0)
    avg_normal /= np.linalg.norm(avg_normal)  # Normalize it

    # Compute the average point on the line
    avg_point = np.mean(points, axis=0)

    # Compute d using the plane equation formula: ax + by + cz + d = 0
    a, b, c = avg_normal
    d = -np.dot(avg_normal, avg_point)

    return np.array([a, b, c, d])  # Return plane coefficients
def compute_average_plane(points, normals):
    """
    Compute an average plane equation ax + by + cz + d = 0 
    for a set of 3D points and their corresponding normal vectors.

    Parameters:
    
    points (np.ndarray): Nx3 array of 3D points along a line.
    normals (np.ndarray): Nx3 array of normal vectors at each point.

        Returns:
        
    np.ndarray: Plane equation coefficients [a, b, c, d]."""
    if len(points) < 3:
        return None  # Not enough points to define a plane

    # Compute the average normal vector
    avg_normal = np.mean(normals, axis=0)
    avg_normal /= np.linalg.norm(avg_normal)  # Normalize it

    # Compute the average point on the line
    avg_point = np.mean(points, axis=0)

    # Compute d using the plane equation formula: ax + by + cz + d = 0
    a, b, c = avg_normal
    d = -np.dot(avg_normal, avg_point)

    return np.array([a, b, c, d])  # Return plane coefficients

import numpy as np
from sklearn.linear_model import RANSACRegressor

def compute_plane_ransac(points, normals, threshold=0.05):
    """
    Compute plane equation using RANSAC for robustness.
    
    Parameters:
    - points: np.ndarray of shape (N, 3) containing 3D points.
    - normals: np.ndarray of shape (N, 3) containing normal vectors.
    - threshold: float, the residual threshold for RANSAC.
    
    Returns:
    - np.ndarray of shape (4,) representing plane equation coefficients [a, b, c, d].
    """
    if len(points) < 3:
        return None

    # Plane equation: ax + by + cz + d = 0
    X = points  # 3D point coordinates (N, 3)

    y = -np.sum(normals *points, axis=1)  # Compute d as dot product
    
    ransac = RANSACRegressor(residual_threshold=threshold)
    ransac.fit(X, y)

    a, b, c = ransac.estimator_.coef_
    d = ransac.estimator_.intercept_
    l = np.array([a, b, c])
    
    return np.array([a, b, c,d])/ (np.linalg.norm(l) + 1e-12)

def get_line_pixels(line, maps):
    """
    Get all pixel coordinates along a line using cv2.line.
    """
    x1, y1 = map(int, line[0])
    x2, y2 = map(int, line[1])
    height, width = maps.shape[:2]

    blank_image = np.zeros((height, width), dtype=np.uint8)
    cv2.line(blank_image, (x1, y1), (x2, y2), color=255, thickness=1)
    
    y_coords, x_coords = np.where(blank_image == 255)
    return list(zip(x_coords, y_coords))

def extract_line_plan(lines, normal_map, world_coordinates):
    """
    Extract plane for each line.
    """
    line_features = []
    for line in lines:
        pixel_coords = get_line_pixels(line, depth_map)

        points, normals = [], []
        for x, y in pixel_coords:
            points.append(world_coordinates[y, x])
            normals.append(normal_map[y, x])

        if len(points) > 2:
            plane = compute_plane_ransac(np.array(points), np.array(normals))
            if plane is not None:
                line_features.append(plane)

    return np.array(line_features, dtype=object)
def plane_distance(plane1, plane2):
    """
    Custom distance function for DBSCAN to compare planes given by (A, B, C, D).
    
    Args:
        plane1: (A1, B1, C1, D1) - Coefficients of the first plane.
        plane2: (A2, B2, C2, D2) - Coefficients of the second plane.
    
    Returns:
        A distance value where lower means more similar planes.
    """

    A1, B1, C1, D1 = plane1
    A2, B2, C2, D2 = plane2

    # Compute normal vectors
    n1 = np.array([A1, B1, C1])
    n2 = np.array([A2, B2, C2])

    # Normalize the normal vectors
    norm_n1 = np.linalg.norm(n1)
    norm_n2 = np.linalg.norm(n2)

    if norm_n1 == 0 or norm_n2 == 0:
        return float('inf')  # Invalid plane

    # Compute cosine of the angle between the normal vectors
    cos_theta = np.dot(n1, n2) / (norm_n1 * norm_n2)

    # Compute the angle difference metric
    angle_difference = 1 - abs(cos_theta)

    # Compute the distance between parallel planes (only if they are parallel)
    if abs(cos_theta) >= 0.99:  
        return abs(D1 - D2) / norm_n1
    else:
        return float('inf')  # Set a base distance for non-parallel planes


def cluster_coplanar_lines(line_planes, min_samples = 5, eps=0.1):
    """Cluster the planar lines (a,b,c,d) which represents the plane equation

    Args:
        line_planes: _description_
        eps (float, optional): hyperprama of DBSCAN. Defaults to 0.1.

    Returns:
        _type_: _description_
    """
    
    if len(line_planes) == 0:
        return []


    feature_matrix = line_planes
    # DBSCAN Clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric=plane_distance).fit(feature_matrix)
    # clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean').fit(feature_matrix)
    # OPTICS clustering inferior to DBSCAN
    clustering = OPTICS(min_samples=min_samples,metric=plane_distance,eps=eps).fit(feature_matrix)
    # clustering = HDBSCAN(min_samples=5,min_cluster_size=5,metric=plane_distance,copy=True).fit(feature_matrix)
    return clustering.labels_

    # from sklearn.cluster import OPTICS

    # optics = OPTICS(min_samples=2, metric=plane_distance)
    # labels = optics.fit_predict(line_planes)
    # return labels

    # import hdbscan

    # hdb = hdbscan.HDBSCAN(min_cluster_size=2, metric=plane_distance)
    # labels = hdb.fit_predict(line_planes)
    # return labels


# Extract features
line_planes = extract_line_plan(pred_lines, normal_map, world_coordinates)

# Apply DBSCAN
coplanar_clusters = cluster_coplanar_lines(line_planes, eps=1.072)

# Visualization Function
def plot_coplanar_lines(lines, labels, image):
    unique_labels = set(labels)
    colors = plt.cm.get_cmap("tab20", len(unique_labels))

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    for idx, line in enumerate(lines):
        label = labels[idx]
        if label == -1:  # Outliers
            color = 'grey'
        else:
            color = colors(label)[:3]  # Get unique color

        plt.plot([line[0, 0], line[1, 0]], [line[0, 1], line[1, 1]], color=color, linewidth=2)
    plt.axis("off")
    plt.show()

# Plot results
plot_coplanar_lines(pred_lines, coplanar_clusters, img)